In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

In [ ]:
%cd '/content/gdrive/My Drive/IDLSProject-main-2'

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import torch
import torch.utils.data
from sklearn.model_selection import train_test_split

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)
    
train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)
print(train.shape, test_sample.shape)


test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

#drop rows
test.drop(test.tail(227).index,inplace = True)
val.drop(val.tail(228).index,inplace = True)
test.shape, val.shape

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape, val_y.shape)

(83000, 502) (21975, 502)
torch.Size([10760, 1]) torch.Size([10760, 1])


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 10000
embed_dims = 32
hidden_size = 100
kernel_size = [3,4,5]

In [3]:
data_dir = './cnn-lstm-amazon-hyperband-trails_10/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./cnn-lstm-amazon-hyperband-trails_10/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [4]:
import torch
import torch.nn as nn
import numpy as np
class Combine(nn.Module):
    def __init__(self,trial,vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len,hidden_size):
        super(Combine, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_size = trial.suggest_int("hidden_size",32,256)
        embed_size = trial.suggest_int("embed_size",16,128)
        n_filters = trial.suggest_int("n_filters",50,200)
        self.embedding = torch.nn.Embedding(vocab_size,embed_size)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],embed_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],embed_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],embed_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dropout = torch.nn.Dropout(dropout)
        #self.cnn = CNN(vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len)
        self.lstm = nn.LSTM(
            input_size=3*filter_size, 
            hidden_size=hidden_size, 
            num_layers=1,
            batch_first=True)
        self.dense = nn.Linear(hidden_size,1)
        self.sig = nn.Sigmoid()
        self.word_dict = None


    def forward(self, x):
        batch_size = x.size(0)
        #print(batch_size)
        embed_input = self.embedding(x)
        embed_input.unsqueeze_(1)
        x1 = torch.tanh(self.dropout(self.conv1(embed_input))).squeeze(3)
        x2 = torch.tanh(self.dropout(self.conv2(embed_input))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(embed_input))).squeeze(3)
        f1 = self.mp1(x1).squeeze(2)
        f2 = self.mp2(x2).squeeze(2)
        f3 = self.mp3(x3).squeeze(2)
        c_out = torch.cat([f1,f2,f3],dim=1)
        r_in = c_out.view(batch_size, 1, -1)
        lstm_out, _ = self.lstm(r_in)
        out = self.dense(lstm_out[:, -1, :])
        return self.sig(out)

In [5]:
import time
import numpy as np 
def objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
    #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Combine(trial,vocab_size, embed_dims, filter_size, kernel_size, dropout, seq_len,hidden_size).to(device).to(device)
    trial.set_user_attr(key="best_model", value=model)
    optimizer = optim.Adam(model.parameters())
    loss_fn = torch.nn.BCELoss()
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_accuracy_epoch = []
    time_train = []
    val_acc = 0
    for epoch in range(epochs):
        start = time.time()
        model.train()
        total_loss = 0
        train_acc = 0
        total = 0
        correct = 0
        for batch in train_dl:         
                batch_X, batch_y = batch
                batch_X = batch_X.to(device)
                batch_y = batch_y.to(device)
                optimizer.zero_grad()
                prediction = model(batch_X)
                loss = loss_fn(prediction, batch_y)
                loss.backward()
                optimizer.step()
                result = np.round(prediction.detach().cpu())
                total_loss += loss.data.item()
                total += batch_y.size(0)
                correct += (result == batch_y.cpu()).sum().item()
                train_acc = correct/total
        train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
        train_acc_epoch.append(np.round(train_acc*100,3))
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                prediction = model(inputs_val)
                loss = loss_fn(prediction, labels_val)
                val_loss.append(loss.item())
                result = np.round(prediction.cpu())
                total += labels_val.size(0)
                correct += (result == labels_val.cpu()).sum().item()
            val_accuracy_epoch.append(np.round((correct/total)*100, 3))
            val_loss_epoch.append(np.round(np.mean(val_loss),3))
            end = time.time() - start
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
            time_train.append(np.round(end,3))
            val_acc = np.round((correct/total)*100, 3)
    write_to_csv(trial.number,epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
    return val_acc


def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    acc = correct/total
    print("Accuracy:", (correct/total)*100)

In [6]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [7]:
import torch.optim as optim
epochs = 10

In [8]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=10,callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 22:10:38,163] A new study created in memory with name: no-name-e9ca4c3b-ce23-4215-b2bf-bdd3b83f36dd


Epoch: 0, BCELoss: 0.3770347839917045
Val Loss: 0.319 	Val Acc: 0.870
Epoch: 1, BCELoss: 0.2815688351057021
Val Loss: 0.304 	Val Acc: 0.878
Epoch: 2, BCELoss: 0.2549089605774147
Val Loss: 0.296 	Val Acc: 0.880
Epoch: 3, BCELoss: 0.23835654782288404
Val Loss: 0.295 	Val Acc: 0.883
Epoch: 4, BCELoss: 0.22433867019612386
Val Loss: 0.302 	Val Acc: 0.880
Epoch: 5, BCELoss: 0.21244155917182025
Val Loss: 0.311 	Val Acc: 0.876
Epoch: 6, BCELoss: 0.19989259494805192
Val Loss: 0.317 	Val Acc: 0.881
Epoch: 7, BCELoss: 0.1887115005269108
Val Loss: 0.321 	Val Acc: 0.880
Epoch: 8, BCELoss: 0.17722200953390405
Val Loss: 0.336 	Val Acc: 0.878
Epoch: 9, BCELoss: 0.1679893698363778


[I 2022-05-14 22:12:57,827] Trial 0 finished with value: 87.593 and parameters: {'optimizer': 'RMSprop', 'lr': 0.04621239626939598, 'dropout': 0.3529977073347015, 'hidden_size': 159, 'embed_size': 46, 'n_filters': 130}. Best is trial 0 with value: 87.593.


Val Loss: 0.359 	Val Acc: 0.876
Epoch: 0, BCELoss: 0.3919129880975528
Val Loss: 0.318 	Val Acc: 0.864
Epoch: 1, BCELoss: 0.290038298698793
Val Loss: 0.300 	Val Acc: 0.876
Epoch: 2, BCELoss: 0.2621110653841352
Val Loss: 0.290 	Val Acc: 0.880
Epoch: 3, BCELoss: 0.24385104807744543
Val Loss: 0.287 	Val Acc: 0.882
Epoch: 4, BCELoss: 0.23104813721883727
Val Loss: 0.293 	Val Acc: 0.880
Epoch: 5, BCELoss: 0.22007579272725136
Val Loss: 0.305 	Val Acc: 0.878
Epoch: 6, BCELoss: 0.209229436014072
Val Loss: 0.306 	Val Acc: 0.879
Epoch: 7, BCELoss: 0.19905318606824401
Val Loss: 0.312 	Val Acc: 0.876
Epoch: 8, BCELoss: 0.19020626887858633
Val Loss: 0.330 	Val Acc: 0.863
Epoch: 9, BCELoss: 0.18178701833429106


[I 2022-05-14 22:20:16,863] Trial 1 finished with value: 84.907 and parameters: {'optimizer': 'Adam', 'lr': 0.0026868749584991517, 'dropout': 0.2792089653207933, 'hidden_size': 160, 'embed_size': 20, 'n_filters': 96}. Best is trial 0 with value: 87.593.


...

Val Loss: 0.342 	Val Acc: 0.881
Epoch: 0, BCELoss: 0.36946415249931525
Val Loss: 0.322 	Val Acc: 0.865
Epoch: 1, BCELoss: 0.27638752920979476
Val Loss: 0.311 	Val Acc: 0.875
Epoch: 2, BCELoss: 0.24461975919015436
Val Loss: 0.303 	Val Acc: 0.877
Epoch: 3, BCELoss: 0.2214996799871505
Val Loss: 0.320 	Val Acc: 0.873
Epoch: 4, BCELoss: 0.19970717419342823
Val Loss: 0.366 	Val Acc: 0.848
Epoch: 5, BCELoss: 0.17699500286211092
Val Loss: 0.384 	Val Acc: 0.852
Epoch: 6, BCELoss: 0.15810062834678823
Val Loss: 0.468 	Val Acc: 0.825
Epoch: 7, BCELoss: 0.142795665460044
Val Loss: 0.524 	Val Acc: 0.816
Epoch: 8, BCELoss: 0.12894764377354048
Val Loss: 0.570 	Val Acc: 0.807
Epoch: 9, BCELoss: 0.11561710736686251


[I 2022-05-14 22:35:51,933] Trial 8 finished with value: 81.32 and parameters: {'optimizer': 'SGD', 'lr': 0.021228249017562446, 'dropout': 0.1878523376435325, 'hidden_size': 235, 'embed_size': 58, 'n_filters': 152}. Best is trial 7 with value: 88.067.


Val Loss: 0.557 	Val Acc: 0.813
Epoch: 0, BCELoss: 0.36304088333644063
Val Loss: 0.312 	Val Acc: 0.867
Epoch: 1, BCELoss: 0.27607109266142527
Val Loss: 0.298 	Val Acc: 0.878
Epoch: 2, BCELoss: 0.24766615581620172
Val Loss: 0.300 	Val Acc: 0.878
Epoch: 3, BCELoss: 0.2256126889891653
Val Loss: 0.303 	Val Acc: 0.878
Epoch: 4, BCELoss: 0.20532431264733333
Val Loss: 0.338 	Val Acc: 0.860
Epoch: 5, BCELoss: 0.18676989924790988
Val Loss: 0.364 	Val Acc: 0.841
Epoch: 6, BCELoss: 0.1666274321557258
Val Loss: 0.482 	Val Acc: 0.817
Epoch: 7, BCELoss: 0.15320602351664958
Val Loss: 0.541 	Val Acc: 0.805
Epoch: 8, BCELoss: 0.1377664949228785
Val Loss: 0.597 	Val Acc: 0.794
Epoch: 9, BCELoss: 0.1254673491708979


[I 2022-05-14 22:38:17,564] Trial 9 finished with value: 78.188 and parameters: {'optimizer': 'RMSprop', 'lr': 0.0001454835719085333, 'dropout': 0.20972925817715501, 'hidden_size': 226, 'embed_size': 59, 'n_filters': 197}. Best is trial 7 with value: 88.067.


Val Loss: 0.702 	Val Acc: 0.782
Study statistics: 
  Number of finished trials:  10
  Number of complete trials:  10
Best trial:
  Value:  88.067
  Params: 
    optimizer: RMSprop
    lr: 4.690148740626518e-05
    dropout: 0.5353777735745225
    hidden_size: 159
    embed_size: 45
    n_filters: 93


In [9]:
torch.save(best_model.state_dict(),"./cnn-lstm-amazon-hyperband_10.pth")

In [10]:
test(best_model, test_dl, epochs)

/ext3/miniconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:691: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:925.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Accuracy: 88.21561338289963
